In [1]:
import re

In [2]:
# To create getter and setter functions for the component information, we'll define two functions:
# 1. get_components: This function will parse the .kicad_pcb file and return a list of components with their details.
# 2. set_components: This function will take a list of components with updated positions/rotations and modify the .kicad_pcb file accordingly.


def get_components(file_data):
    """
    Parses the .kicad_pcb file data and extracts component information along with the designator.

    Args:
    file_data (str): Contents of the .kicad_pcb file.

    Returns:
    list of dicts: List of components with details (uuid, position, rotation, designator).
    """
    # Regex to match component footprint and extract UUID, position, rotation, and designator
    component_regex = re.compile(
        r'\(footprint\s+"[^"]+"\s+\(layer\s+"[^"]+"\)\s*\(tstamp\s+([^\s]+)\)\s*\(at\s+([0-9.-]+)\s+([0-9.-]+)\s*([0-9.-]*)\).*?\(fp_text\s+reference\s+"([^"]+)"',
        re.MULTILINE | re.DOTALL
    )

    components = component_regex.findall(file_data)
    parsed_components = [
        {'uuid': uuid, 'x': float(x), 'y': float(y), 'rotation': float(rot) if rot else 0.0, 'designator': designator}
        for uuid, x, y, rot, designator in components
    ]
    return parsed_components

def set_components(file_data, updated_components):
    """
    Updates the .kicad_pcb file data with new positions and rotations for components.
    Also updates pad rotations by adding the component's rotation to the original pad rotation.

    Args:
    file_data (str): Original contents of the .kicad_pcb file.
    updated_components (list of dicts): List of components with updated details (uuid, new position, new rotation).

    Returns:
    str: Updated contents of the .kicad_pcb file.
    """
    for component in updated_components:
        uuid = component['uuid']
        new_x = component['x']
        new_y = component['y']
        new_rot = component['rotation']

        # Update the position and rotation of the component in the file data
        file_data = re.sub(
            rf'(\(footprint\s+"[^"]+"\s+\(layer\s+"[^"]+"\)\s*\(tstamp\s+{uuid}\)\s*)\(at\s+([0-9.-]+)\s+([0-9.-]+)\s*([0-9.-]*)\)',
            rf'\1(at {new_x} {new_y} {new_rot})',
            file_data
        )

        # Regex for finding and updating pad rotations
        pad_regex = rf'(\(pad\s+[^\(]*\(at\s+([0-9.-]+)\s+([0-9.-]+)\s*)([0-9.-]+)(\)\s+\(size)'
        def pad_rotation_match(match):
            original_pad_rot = float(match.group(4))
            new_pad_rot = original_pad_rot + new_rot
            return f"{match.group(1)}{new_pad_rot}{match.group(5)}"
        file_data = re.sub(pad_regex, pad_rotation_match, file_data, flags=re.MULTILINE)

    return file_data



In [3]:
# import pcb file
project_path = "/Users/narayanpowderly/Documents/atopile-workspace/blurp/elec/layout/blurp.kicad_pcb"
kicad_pcb_data = open(project_path, 'r').read()

In [4]:
components = get_components(kicad_pcb_data)


# made a pandas dataframe to store the component data, splitting position into x and y
import pandas as pd
df = pd.DataFrame(components)

df


,uuid,x,y,rotation,designator
0,0155b31d-807e-4709-a708-74276523f91a,142.621815,47.828991,180.0,R4
1,028c61e6-fe97-45a8-a968-d89920e62685,139.676173,50.773032,-90.0,C2
2,050c7d0f-cc53-4f21-b306-6f6051e3f48c,130.979847,43.968404,180.0,C3
3,0cbd6dd5-a455-4811-9695-df40dcde7434,130.985499,44.943192,180.0,C8
4,1986d113-0e9a-4e89-a3a6-2ee70afdb020,129.540000,36.317206,-90.0,U3
5,29275dbe-0c29-496f-9e53-75ce672a6446,140.002139,37.750408,90.0,C11
6,30702a2b-78ba-4d57-9fe9-958f1537fb50,133.369003,51.635969,90.0,C1
7,3b19cd19-0780-4dbc-aef6-a10e4453f9ca,138.061220,37.770408,90.0,R2
8,3e9e5fc4-befc-42a0-afca-2695b7c91066,137.086528,37.750408,90.0,C7
9,4b318026-11db-4cdf-b5e9-22ef8c4d274d,139.037051,37.770408,-90.0,R3


In [7]:
# Let's say we want to update the position and rotation of the first component
components = get_components(kicad_pcb_data)

components[0]['x'] += 10  # Move the x position by 10mm
# components[0]['rotation'] = 45  # Rotate the component to 45 degrees
# rotate all components by 35 degrees
for component in components:
    component['rotation'] += 35

# Updating the file with the new component data
updated_file_data = set_components(kicad_pcb_data, components)

# Write the updated file data to a new file
with open("rotated.kicad_pcb", 'w') as f:
    f.write(updated_file_data)

# Display the first part of the updated file data to verify the change
# print(updated_file_data[:1000])  # Print first 1000 characters of the updated file data
# components
# updated_components = get_components(updated_file_data)

# updated_components
# updated_file_data